In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
import seaborn as sns

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_100deg_uniform.npy')
pm_data_noerr = np.load('npy/pm_LAE_100deg_uniform.npy')
f_name = 'Source_cats/Source_cat_LAE_100deg_uniform.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/median_errors.npy')

pm_err = (np.ones(pm_data.shape).T * errors).T

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
cont_est, cont_err = stack_estimation(pm_data, pm_err, 13, 5, w_central)

In [ ]:
np.random.seed(22)
for mock_n in np.random.randint(0, pm_data.shape[1], 20):
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots(figsize = (12, 8))
    ax.plot(mock['w_Arr'], mock['SEDs_no_line'][mock_n], c='orange', lw=2, zorder=-1)
    ax.plot(mock['w_Arr'], mock['SEDs_no_IGM'][mock_n], c='green', lw=2, zorder=-1)
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], c='gray', fmt='.')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4],
                xerr=bb_fwhm[-4]/2, yerr=errors[-4],
                fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], yerr=errors[-3],
                xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], yerr=errors[-2],
                xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], yerr=errors[-1],
                xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    f_line_cont = NB_synthetic_photometry(
        mock['SEDs_no_line'][mock_n], mock['w_Arr'],
        (mock['redshift_Lya_Arr'][mock_n] + 1) * 1215.67,
        11
    )

    ax.plot(
        (mock['redshift_Lya_Arr'][mock_n] + 1) * 1215.67, f_line_cont,
        marker='.', markersize=15, color='black'
    )

    IGM_T = IGM_TRANSMISSION(w_central[13])
    ax.errorbar(
        w_central[13], cont_est[mock_n]*IGM_T, marker='^', markersize=11,
        yerr = cont_err[mock_n], elinewidth=3, capsize=3, capthick=3
    )

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)
    ax.set_ylim((
        np.amin(mock['SEDs_no_line'][mock_n]) * (1 - 0.6),
        np.amax(mock['SEDs_no_line'][mock_n]) * (1 + 0.6)
        ))

    plt.show()
    print(mock['EW_Arr'][mock_n])